In [1]:
from core.model import GBNEncoder, LNClassifier
from core.sim_metric import EDSim, SDPSim, CosSim
import os
import torch

opt = {'data_dir': '../KnowledgeGraph_materials/data_kg/bootstrapnet_data/boot_pretrain_data_revised/',
       'output_model_file': './models/test', 
       'input_model_file': './models/210910_selfSupervised', 
       'sim_metric': CosSim(), 'k_hop': 2, 
       'nl_weight': 0.01, 
       'local': False, 
       'n_layer': 3, 
       'dropout': 0.1, 
       'negative_slope': 0.2, 
       'bias': False, 
       'device': torch.device(type='cuda', index=0), 
       'cpu': False, 'seed': 1, 
       'n_epoch': 100, 
       'optimizer': 'adam', 
       'lr': 0.001, 
       'decay': 0.001, 
       'max_grad_norm': 1.0, 
       'feature_type': 'random', 
       'feature_dim': 50, 
       'edge_feature_dim': 5}

In [3]:
encoder = GBNEncoder(opt)

encoder.load_state_dict(torch.load(opt['input_model_file']+'.pth'))

encoder.eval()

GBNEncoder(
  (node_nns): ModuleList(
    (0): GNNConv(
      (depth_encoder): EmbeddingEncodingLayer(
        (encoding): Embedding(3, 5)
      )
      (edge_encoder): EmbeddingEncodingLayer(
        (encoding): Embedding(4, 5)
      )
      (attn_layer): GraphAttentionLayer(
        (w_q): Linear(in_features=50, out_features=50, bias=False)
        (w_k): Linear(in_features=60, out_features=50, bias=False)
        (w_v): Linear(in_features=50, out_features=50, bias=False)
        (attention): FlattenScaledDotProduct(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (w_res): Linear(in_features=50, out_features=50, bias=False)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (1): GNNConv(
      (depth_encoder): EmbeddingEncodingLayer(
        (encoding): Embedding(3, 5)
      )
      (edge_encoder): EmbeddingEncodingLayer(
        (encoding): Embedding(4, 5)
      )
      (attn_layer): GraphAttentionLayer(
        (w_q): Linear(in_features=50, out_feat